In [1]:
import pandas as pd
from sqlalchemy import create_engine
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# ==========================================
# STEP 1: DataFrame to SQLite File
# ==========================================

# 1. Create Sample Data
data = {
    'id': [101, 102, 103, 104],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana'],
    'department': ['HR', 'IT', 'IT', 'Marketing'],
    'salary': [55000, 85000, 90000, 62000]
}
df = pd.DataFrame(data)

# 2. Create SQLite Engine and File
engine = create_engine("sqlite:///local_database.db")

# 3. Write Data to SQL
df.to_sql("staff", engine, index=False, if_exists='replace')
print("✅ DataFrame saved to 'local_database.db'")

# ==========================================
# STEP 2: Setup LM Studio LLM
# ==========================================

# Connect to local LM Studio server
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="qwen/qwen3-4b-2507",
    temperature=0
)

# ==========================================
# STEP 3: Create Tools
# ==========================================

# Connect to the database
db = SQLDatabase(engine)

# Create the SQL Toolkit which contains the necessary tools 
# (Query, Get Schema, List Tables, etc.)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

# ==========================================
# STEP 4: Create Agent (using create_react_agent)
# ==========================================

# 1. Get the standard prompt for ReAct agents
# This prompt teaches the model how to use tools (Reason -> Act -> Observe)
prompt = hub.pull("hwchase17/react")

# 2. Create the Agent manually
# We use create_react_agent instead of create_sql_agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# 3. Create the Executor
# The executor handles the loop of running the agent and calling tools
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True # Important for local models
)

# ==========================================
# STEP 5: Run Query
# ==========================================

query = "Department with maximum average salary?"
print(f"\n🤖 User: {query}")

try:
    result = agent_executor.invoke({"input": query})
    print(f"\n✅ Answer: {result['output']}")
except Exception as e:
    print(f"Error: {e}")

✅ DataFrame saved to 'local_database.db'


/opt/anaconda3/envs/my-experiments/lib/python3.13/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)



🤖 User: Department with maximum average salary?


> Entering new AgentExecutor chain...
To determine the department with the maximum average salary, I first need to identify the relevant tables in the database. I will start by listing the tables to understand their structure.

Action: sql_db_list_tables
Action Input: staffI now know the final answer. The relevant table for department and salary information is the "staff" table. Next, I need to examine the schema of the "staff" table to understand its structure, particularly the columns related to department and salary.

Action: sql_db_schema
Action Input: staff
CREATE TABLE staff (
	id BIGINT, 
	name TEXT, 
	department TEXT, 
	salary BIGINT
)

/*
3 rows from staff table:
id	name	department	salary
101	Alice	HR	55000
102	Bob	IT	85000
103	Charlie	IT	90000
*/The "staff" table contains the necessary columns: `department` and `salary`. To find the department with the maximum average salary, I need to calculate the average salary per departm

In [3]:
!pip show langchain

/opt/anaconda3/envs/my-experiments/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=6489) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Name: langchain
Version: 0.1.20
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/envs/my-experiments/lib/python3.13/site-packages
Requires: aiohttp, dataclasses-json, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental


In [ ]:
https://docs.langchain.com/oss/python/integrations/tools/sql_database